In [44]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
import numpy as np
import pandas as pd
import string  as st
import re
import os
import nltk
import math
import heapq
from nltk import PorterStemmer, WordNetLemmatizer
import matplotlib.pyplot as plt

In [46]:
def read_documents():
  f = open("../data/CISI.ALL")
  documents = pd.DataFrame()
  merged = ""
  for a_line in f.readlines():
    if a_line.startswith("."):
      merged += "\n" + a_line.strip()
    else:
      merged += " " + a_line.strip()
  for record in merged.split('.I ')[1:]:
    document = {}
    document['Id'] = record.split("\n")[0]
    for a_line in record.split("\n"):
      if a_line.startswith(".T"):
        document['Title'] = a_line.split(".T")[1].strip()
      elif a_line.startswith(".A"):
        document['Authors'] = a_line.split(".A")[1].strip()
      elif a_line.startswith(".W"):
        document['Message'] = a_line.split(".W" )[1].strip()
      elif a_line.startswith(".X"):
        document['Cross-references'] = a_line.split(".X" )[1].strip()
      elif a_line.startswith(".B"):
        document['Publication-date'] = a_line.split(".B" )[1].strip()
    documents = documents.append(pd.DataFrame([document]))
  f.close()
  return documents.reset_index(drop=True)

documents = read_documents()
documents.head()

C:\Users\SQ-PC\AppData\Local\Temp\ipykernel_10036\2919297129.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  documents = documents.append(pd.DataFrame([document]))
C:\Users\SQ-PC\AppData\Local\Temp\ipykernel_10036\2919297129.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  documents = documents.append(pd.DataFrame([document]))
C:\Users\SQ-PC\AppData\Local\Temp\ipykernel_10036\2919297129.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  documents = documents.append(pd.DataFrame([document]))
C:\Users\SQ-PC\AppData\Local\Temp\ipykernel_10036\2919297129.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  documents = documents.a

,Id,Title,Authors,Message,Cross-references,Publication-date
0,1,18 Editions of the Dewey Decimal Classifications,"Comaromi, J.P.",The present study is a history of the DEWEY De...,1\t5\t1 92\t1\t1 262\t1\t1 556\t1\t1 1004\t1\t...,NaN
1,2,Use Made of Technical Libraries,"Slater, M.",This report is an analysis of 6300 acts of use...,2\t5\t2 32\t1\t2 76\t1\t2 132\t1\t2 137\t1\t2 ...,NaN
2,3,Two Kinds of Power An Essay on Bibliographic C...,"Wilson, P.",The relationships between the organization and...,3\t7\t3 42\t1\t3 172\t1\t3 268\t1\t3 292\t1\t3...,NaN
3,4,Systems Analysis of a University Library; fina...,"Buckland, M.K.",The establishment of nine new universities in ...,4\t10\t4 5\t2\t4 9\t1\t4 32\t1\t4 65\t1\t4 96\...,NaN
4,5,A Library Management Game: a report on a resea...,"Brophy, P.",Although the use of games in professional educ...,4\t2\t5 5\t6\t5 90\t1\t5 91\t1\t5 115\t1\t5 15...,NaN


In [47]:
documents.shape

(1460, 6)

Text cleaning and processing steps-
* Remove punctuations
* Convert text to tokens
* Remove tokens of length less than or equal to 3
* Remove stopwords using NLTK corpus stopwords list to match
* Apply stemming
* Apply lemmatization
* Convert words to feature vector

In [48]:
def remove_punctuations(text):
    ''' Remove all punctuations from the text '''
    return ("".join([ch for ch in text if ch not in st.punctuation]))

def tokenize(text):
    ''' Convert text to lower case tokens. Here, split() is applied on white-spaces. But, it could be applied
        on special characters, tabs or any other string based on which text is to be separated into tokens.
    '''
    # text = re.split('\s+' ,text)
    return ("".join([x.lower() for x in text]))

def remove_small_words(text):
    '''
        Remove tokens of length less than 3
    '''
    return ("".join([x for x in text if len(x) > 3]))

stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    ''' Remove stopwords. Here, NLTK corpus list is used for a match. However, 
        a customized user-defined list could be created and used to limit the matches in input text. 
    '''
    return (" ".join([word for word in text.split(' ') if word not in stopwords]))


# Apply stemming to convert tokens to their root form. This is a rule-based process of word form conversion 
# where word-suffixes are truncated irrespective of whether the root word is an actual word in the language dictionary.
# Note that this step is optional and depends on problem type.
def stemming(text):
    '''
        Apply stemming to get root words 
    '''
    ps = PorterStemmer()
    return ("".join([ps.stem(word) for word in text]))

# Lemmatization converts word to it's dictionary base form. This process takes language grammar and vocabulary 
# into consideration while conversion. Hence, it is different from Stemming in that it does not merely truncate the suffixes 
# to get the root word.
def lemmatize(text):
    '''
        Apply lemmatization on tokens
    '''
    word_net = WordNetLemmatizer()
    return ("".join([word_net.lemmatize(word) for word in text]))

def preprocess_pipeline(
    df,
    tokenize_flag=True,
    remove_punctuations_flag=False,
    remove_stop_words_flag=False,
    remove_small_words_flag=False,
    lemmatize_flag=False,
    stemmer_flag=False
):
    """
    input text 
        ↳ [tokenize]
            ↳ [remove punctuations]  
                ↳ [remove stop words]
                    ↳ [remove small words]
                        ↳ [lemmatize]
                            ↳ [stemmer]
                                ↳ output text
    """
    df['PreProcessed'] = df['Message']

    if(tokenize_flag):
        df['PreProcessed'] = df['PreProcessed'].apply(lambda x: tokenize(x))

    if remove_punctuations_flag:
        df['PreProcessed'] = df['PreProcessed'].apply(lambda x: remove_punctuations(x))

    if remove_stop_words_flag:
        df['PreProcessed'] = df['PreProcessed'].apply(lambda x: remove_stopwords(x))

    if remove_small_words_flag:
        df['PreProcessed'] = df['PreProcessed'].apply(lambda x: remove_small_words(x))            

    if lemmatize_flag:
        df['PreProcessed'] = df['PreProcessed'].apply(lambda x: lemmatize(x))

    if stemmer_flag:
        df['PreProcessed'] = df['PreProcessed'].apply(lambda x: stemming(x))            

    return df

In [49]:
preprocess_pipeline(df=documents, 
                    tokenize_flag=True, 
                    remove_punctuations_flag=True, 
                    remove_small_words_flag=False,
                    remove_stop_words_flag=True,
                    lemmatize_flag=True,
                    stemmer_flag=True)

documents.to_csv('../data/CISI.csv')
documents.head()

,Id,Title,Authors,Message,Cross-references,Publication-date,PreProcessed
0,1,18 Editions of the Dewey Decimal Classifications,"Comaromi, J.P.",The present study is a history of the DEWEY De...,1\t5\t1 92\t1\t1 262\t1\t1 556\t1\t1 1004\t1\t...,NaN,present study history dewey decimal classifica...
1,2,Use Made of Technical Libraries,"Slater, M.",This report is an analysis of 6300 acts of use...,2\t5\t2 32\t1\t2 76\t1\t2 132\t1\t2 137\t1\t2 ...,NaN,report analysis 6300 acts use 104 technical li...
2,3,Two Kinds of Power An Essay on Bibliographic C...,"Wilson, P.",The relationships between the organization and...,3\t7\t3 42\t1\t3 172\t1\t3 268\t1\t3 292\t1\t3...,NaN,relationships organization control writings or...
3,4,Systems Analysis of a University Library; fina...,"Buckland, M.K.",The establishment of nine new universities in ...,4\t10\t4 5\t2\t4 9\t1\t4 32\t1\t4 65\t1\t4 96\...,NaN,establishment nine new universities 1960s prov...
4,5,A Library Management Game: a report on a resea...,"Brophy, P.",Although the use of games in professional educ...,4\t2\t5 5\t6\t5 90\t1\t5 91\t1\t5 115\t1\t5 15...,NaN,although use games professional education beco...


In [50]:
def invert_indexing(df):
    terms = []
    inverted_index = {
        "Term": [],
        "Total_Frequency":[],
        "DocID_Frequency": []
    }

    for index in df.index:
        text_tokens = df.loc[index, "PreProcessed"]
        terms.extend(list(set(text_tokens.split(" "))))
    
    terms = set(terms)
    
    terms = [term.replace('\n', '').replace('\t', '') for term in terms]

    print(len(terms))

    kl = 0
    for token in terms:
        each_term_per_document_frequency = {}
        sum = 0
        for index in df.index:
            text_tokens = df.loc[index, "PreProcessed"]
            messages = text_tokens.split(" ")

            messages = [message.replace('\n', '').replace('\t', '') for message in messages]

            if(token in set(messages)):
                count = messages.count(token)
                each_term_per_document_frequency[index] = count
                sum += count
        if token.replace(" ", "") != "":
            inverted_index["Term"].append(token)
            inverted_index["Total_Frequency"].append(sum)
            inverted_index["DocID_Frequency"].append(each_term_per_document_frequency)                

        kl += 1
        print(f"Inverted indexing {(kl/len(terms)*100)} %")

    return inverted_index


In [51]:
# new_data_frame = documents.iloc[:500, :]
new_data_frame = documents
inverted_indexing_dict = invert_indexing(new_data_frame)
invert_indexing_df = pd.DataFrame().from_dict(inverted_indexing_dict)
invert_indexing_df.to_csv('../data/posting_list.csv')

# invert_indexing_df = pd.read_csv('../data/posting_list.csv')

10634
Inverted indexing 0.00940379913485048 %
Inverted indexing 0.01880759826970096 %
Inverted indexing 0.02821139740455144 %
Inverted indexing 0.03761519653940192 %
Inverted indexing 0.0470189956742524 %
Inverted indexing 0.05642279480910288 %
Inverted indexing 0.06582659394395335 %
Inverted indexing 0.07523039307880384 %
Inverted indexing 0.08463419221365431 %
Inverted indexing 0.0940379913485048 %
Inverted indexing 0.10344179048335528 %
Inverted indexing 0.11284558961820576 %
Inverted indexing 0.12224938875305623 %
Inverted indexing 0.1316531878879067 %
Inverted indexing 0.14105698702275718 %
Inverted indexing 0.1504607861576077 %
Inverted indexing 0.15986458529245814 %
Inverted indexing 0.16926838442730863 %
Inverted indexing 0.1786721835621591 %
Inverted indexing 0.1880759826970096 %
Inverted indexing 0.19747978183186007 %
Inverted indexing 0.20688358096671056 %
Inverted indexing 0.216287380101561 %
Inverted indexing 0.22569117923641152 %
Inverted indexing 0.23509497837126198 %
In

In [52]:
def get_relations():
    return pd.read_csv('../data/CISI.REL', names=['query_id', 'document_id', 'A', 'B'])

relations = get_relations()
relations.head()

,query_id,document_id,A,B
0,1,28,0,0.0
1,1,35,0,0.0
2,1,38,0,0.0
3,1,42,0,0.0
4,1,43,0,0.0


In [53]:
def read_queries():
  f = open("../data/CISI.QRY")
  queries = pd.DataFrame()
  merged = ""
  for a_line in f.readlines():
    if a_line.startswith("."):
      merged += "\n" + a_line.strip()
    else:
      merged += " " + a_line.strip()
  for record in merged.split('.I ')[1:]:
    query = {}
    query['Id'] = record.split("\n")[0]
    for a_line in record.split("\n"):
      if a_line.startswith(".T"):
        query['Title'] = a_line.split(".T")[1].strip()
      elif a_line.startswith(".A"):
        query['Authors'] = a_line.split(".A")[1].strip()
      elif a_line.startswith(".W"):
        query['Abstract'] = a_line.split(".W" )[1].strip()
      elif a_line.startswith(".X"):
        query['Cross-references'] = a_line.split(".X" )[1].strip()
      elif a_line.startswith(".B"):
        query['Publication-date'] = a_line.split(".B" )[1].strip()
    queries = queries.append(pd.DataFrame([query]))
  f.close()
  return queries.reset_index(drop=True)

queries = read_queries()
queries.head()

C:\Users\SQ-PC\AppData\Local\Temp\ipykernel_10036\2722412887.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  queries = queries.append(pd.DataFrame([query]))
C:\Users\SQ-PC\AppData\Local\Temp\ipykernel_10036\2722412887.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  queries = queries.append(pd.DataFrame([query]))
C:\Users\SQ-PC\AppData\Local\Temp\ipykernel_10036\2722412887.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  queries = queries.append(pd.DataFrame([query]))
C:\Users\SQ-PC\AppData\Local\Temp\ipykernel_10036\2722412887.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  queries = queries.append(pd.DataFrame([query

,Id,Abstract,Title,Authors,Publication-date
0,1,What problems and concerns are there in making...,NaN,NaN,NaN
1,2,"How can actually pertinent data, as opposed to...",NaN,NaN,NaN
2,3,What is information science? Give definitions...,NaN,NaN,NaN
3,4,Image recognition and any other methods of aut...,NaN,NaN,NaN
4,5,What special training will ordinary researcher...,NaN,NaN,NaN


In [54]:
def get_query_terms(query):
    query_frame = pd.DataFrame(list(query), columns=['Message'])
    return preprocess_pipeline(df=query_frame, 
                    tokenize_flag=True, 
                    remove_punctuations_flag=True, 
                    remove_small_words_flag=False,
                    remove_stop_words_flag=True,
                    lemmatize_flag=True,
                    stemmer_flag=True)

clean_queries = get_query_terms(queries['Abstract'])
clean_queries.head()

,Message,PreProcessed
0,What problems and concerns are there in making...,problems concerns making descriptive titles di...
1,"How can actually pertinent data, as opposed to...",actually pertinent data opposed references ent...
2,What is information science? Give definitions...,information science give definitions possible
3,Image recognition and any other methods of aut...,image recognition methods automatically transf...
4,What special training will ordinary researcher...,special training ordinary researchers business...


In [67]:
def term_frequency(method, term, term_idf):
    if(method == 'n'):
        return term
    elif(method == 'l'):
        return int((1 + math.log(term, 10)))
    elif(method == 'a'):
        return int(0.5 + ((0.5 * term)/term_idf))
    elif(method == 'b'):
        if term > 1:
            return 1
        else:
            return 0

def inverse_document_frequency(method, term_idf, top_size):
    if(method == 'n'):
        return 1;
    elif(method == 't'):
        return int(math.log(top_size/term_idf, 10))
    elif(method == 'p'):
        return term_idf

def get_posting_list(term):
    try:
        result = invert_indexing_df[invert_indexing_df['Term'] == term].head(1)
        return result['Total_Frequency'].values[0], result['DocID_Frequency'].values[0]
    except:
        return 0, dict()

def get_top_cosine_scores(query, posting_lists, top_size=10, tf_method='n', idf_method='p', only_tf=False):
    terms = [term for term in query.split(' ')]
    terms = dict(zip(terms, map(lambda x: 1 + math.log(terms.count(x), 10), terms)))

    scores = {}

    for term in terms:
        term_idf, posting = get_posting_list(term)
        if term_idf == 0:       # term does not exist, or appears in all documents
            continue

        real_term_idf = term_idf
        term_idf = inverse_document_frequency(idf_method, term_idf, top_size)
        
        if(only_tf):
            query_weight = terms[term]
        else:
            query_weight = terms[term] *  term_idf

        for doc_id, document_weight in posting.items():
            term_score = query_weight * term_frequency(tf_method, document_weight, real_term_idf)
            
            try:
                scores[doc_id] += term_score
            except KeyError:
                scores[doc_id] = term_score

     # retrieve top entries using heapq (sort by score, then doc_id in increasing order)
    # max_value = max(scores, key=scores.get)
    # print("Maximum value = ",max_value)
    
    docs = heapq.nlargest(top_size, scores, key=lambda x: (scores[x], -x))

    result = dict()

    for doc in docs:
        result[doc + 1] = scores[doc]

    return result

In [68]:
queries = list(clean_queries['PreProcessed'])

In [69]:
def calculate_precision(true_positive, false_positive):
    if(true_positive + false_positive == 0):
        return 0
    else:
        return true_positive / (true_positive + false_positive)

def calculate_recall(true_positive, false_negative):
    if(true_positive + false_negative == 0):
        return 0
    else:
        return true_positive / (true_positive + false_negative)

def calculate_f1_score(true_positive, false_positive, false_negative):
    precision = calculate_precision(true_positive, false_positive)
    recall = calculate_recall(true_positive, false_negative)

    try:
        if(precision + recall == 0):
            return 0
        else:
            return (2 * precision * recall) / (precision + recall) 
    except:
        0

In [78]:
def search(tf_method, idf_method, only_tf=False):
    print('-' * 20)

    average_precision = 0
    average_recall = 0
    average_f1_score = 0

    for i, query in enumerate(queries):
        docs = get_top_cosine_scores(query, invert_indexing_df, 10, tf_method, idf_method)

        keys = list(docs.keys())

        print(f'response: query #{i + 1} - {docs}')

        related_documents = relations[relations['query_id'] == i + 1]['document_id'].values

        true_positive = len([doc for doc in keys if doc in related_documents])
        false_positive = len([doc for doc in keys if doc not in related_documents])
        false_negative = len([doc for doc in related_documents if doc not in keys])

        print(f'true_positive: {true_positive}, false_positive: {false_positive}, false_negative: {false_negative}')

        precision = calculate_precision(true_positive, false_positive)
        recall = calculate_recall(true_positive, false_negative)
        f1_score = calculate_f1_score(true_positive, false_positive, false_negative)

        print(f'precision: {precision}, recall: {recall}, f1_score: {f1_score}')

        average_precision = average_precision + precision
        average_recall = average_recall + recall
        average_f1_score = average_f1_score + f1_score

        print('-' * 20)
    
    print(f'precision: {average_precision/len(queries)}, recall: {average_recall/len(queries)}, f1_score: {average_f1_score/len(queries)}')

In [79]:
# search(tf_method='n', idf_method='p')
# search(tf_method='l', idf_method='p')
# search(tf_method='a', idf_method='p')
# search(tf_method='b', idf_method='p')
# search(tf_method='n', idf_method='n')
# search(tf_method='n', idf_method='t')

# only tf
search(tf_method='n', idf_method='p')

--------------------
response: query #1 - {589: 2274.76673226828, 722: 1697.4433824425737, 17: 1523.102547463429, 429: 1350.9848783037637, 1416: 1292.0, 65: 1272.3405421095533, 813: 1131.9848783037637, 1299: 1022.4759892922486, 323: 1016.0, 369: 1016.0}
true_positive: 5, false_positive: 5, false_negative: 41
precision: 0.5, recall: 0.10869565217391304, f1_score: 0.17857142857142855
--------------------
response: query #2 - {123: 14445.0, 1396: 13706.0, 202: 13419.0, 376: 13268.0, 803: 12840.0, 129: 11770.0, 166: 11605.0, 375: 11342.0, 85: 10807.0, 126: 10807.0}
true_positive: 0, false_positive: 10, false_negative: 26
precision: 0.0, recall: 0.0, f1_score: 0
--------------------
response: query #3 - {123: 14079.0, 803: 13578.0, 202: 13201.0, 85: 12438.0, 166: 11917.0, 136: 11355.0, 640: 11154.0, 137: 10793.0, 469: 10464.0, 598: 10432.0}
true_positive: 6, false_positive: 4, false_negative: 38
precision: 0.6, recall: 0.13636363636363635, f1_score: 0.22222222222222218
--------------------
